[PEP 519](https://www.python.org/dev/peps/pep-0519/#proposal) discusses a really cool feature that controls how the `open` and `pathlib` treat an `object`. I originally found out about this on [Brett Cannon's twitter](https://twitter.com/brettsky/status/1063258648237891584).  

The `os` modules contains a base class `os.PathLike`.

In [1]:
    import os, dataclasses, pathlib, types

The `os.PathLike` subclasses allows an author to define a `__fspath__` attributes that assists in accessing the file `object`.

Consider a test class `PathParty` where `__fspath__` returns a path defined by the `dataclasses`.

In [2]:
    @dataclasses.dataclass
    class PathParty(os.PathLike):
        path: str = '2018-11-16-fspath.ipynb' #<-- the name of this blog post.
        def __fspath__(self): return self.path

The `party`  instance of `PathParty` may now be used with `open` and `pathlib`.

In [3]:
    party = PathParty()
    with open(party) as file: 
        assert file.read() == pathlib.Path(party).read_text()

Notebooks may be reused as modules or data.  The <code>__fspath__</code> property could ease the access of data in notebooks.  

`ModuleOpenable` is both a `types.ModuleType` and `os.PathLike`.  Adding the `os.PathLike` attribute simplifies access to underlying notebook data. 

In [4]:
    class ModuleOpenable(types.ModuleType, os.PathLike):
        def __fspath__(self): return self.__file__

We'll __import__ this specific post as a module.

In [5]:
    import _018_11_16_fspath as a_notebook

This notebook module is the same as one used in our `party` demonstration.

In [18]:
    assert pathlib.Path(a_notebook.__file__) == pathlib.Path(party.path).absolute()

We'll force `a_notebook` to be an `openable_module` that is of the `type` `ModuleOpenable`.

In [19]:
    openable_module = ModuleOpenable(a_notebook.__name__)
    openable_module.__dict__.update(a_notebook.__dict__)

Now `openable_module` can be used directly with `open`, this is equal to the previous demonstrations.

In [21]:
    with open(openable_module) as opened_module, open(party) as file:
        assert opened_module.read() == file.read() == pathlib.Path(party).read_text()

The <code>pandas.read_*</code> can access `os.PathLike` objects meaning that a `ModuleOpenable` can be used directly with `pandas`. 

In [24]:
    import pandas
    pandas.read_json(openable_module, typ=pandas.Series).to_frame().T

,cells,metadata,nbformat,nbformat_minor
0,"[{'cell_type': 'markdown', 'metadata': {}, 'so...","{'kernelspec': {'display_name': 'Python 3', 'l...",4,2


This will be useful!  I'm stoked. 

I am considering adding `os.PathLike` to `importnb`.